# Background on Taproot

## Where does "taproot" come from?

Taproot is a series of bitcoin BIPS (bitcoin improvement proposals) that were merged into bitcoin-core in 2021.

It was most recent bitcoin soft-fork and it went live in November 2021.

The BIPS can be found on github. They are as follows:

- BIP340, Schnorr Signatures https://github.com/bitcoin/bips/blob/master/bip-0340.mediawiki
- BIP341, Taproot Segwit v1 https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki
- BIP342, Validating Taproot Spends https://github.com/bitcoin/bips/blob/master/bip-0342.mediawiki
- BIP350, Bech32m https://github.com/bitcoin/bips/blob/master/bip-0350.mediawiki

## What did "taproot" change?

Taproot added and modified many things in bitcoin. Including

- A new signature algorithm (Schnorr)
- A new version of Script opcodes (Segwit v1)
- A new bitcoin address checksum (bech32m)
- Removed some opcodes we know and love (OP_CHECKMULTISIG)
- Added a new opcode (OP_CHECKSIGADD)
- Added a new scripthash algorithm

# Finding Taproot in the Wild

Let's go to mempool.space and find some taproot addresses in the wild!



## How to spot a taproot address

The bech32 address has a `p` immediately following the `1`

Find a taproot address on mempool dot space.

What's the "Type" for this address?


Copy paste the "ScriptPubKey (HEX)" for the address you found into the pyblock below.

In [1]:
scriptpubkey = "51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953"

In [2]:
print(scriptpubkey)

51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953


In [3]:
len(scriptpubkey) // 2

34

This is a series of opcodes. If you pass this to `bitcoin-cli decodescript` what does it print back as the asm?

In [4]:
!bitcoin-cli -regtest decodescript 51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953

{
  "asm": "1 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953",
  "desc": "addr(bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl)#95u636fj",
  "address": "bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl",
  "type": "witness_v1_taproot"
}


In [5]:
output = {
  "asm": "1 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953",
  "desc": "addr(bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl)#95u636fj",
  "address": "bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl",
  "type": "witness_v1_taproot"
}

In [6]:
print(output['asm'])

1 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953


In [7]:
output['asm'].split(" ")

['1', '3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953']

In [8]:
len(output['asm'].split(" ")[1]) // 2

32

In [9]:
print(output['type'])

witness_v1_taproot


In [10]:
x_bytes = bytes.fromhex(output['asm'].split(' ')[1])
print('hex:\t\t', x_bytes.hex())

x_val = int.from_bytes(x_bytes, 'big')
print('int val:\t', x_val)

hex:		 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953
int val:	 25792158117016797743484614046835601668965634494428223733945610235101691103571


## Finding the secp256k1 pubkey from a P2TR address

Now that we have the x-only pubkey, let's figure out what point that is on the secp256k1 curve.

We can do that using `lift_x_pubkey`. This takes an integer and returns a point.

In my case, I'll use `coincurve`, a python library for expressing + doing math with public keys.

In [80]:
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
def lift_x(x_value):
    assert x_value < p
    # find y squared (y2 = x^3 + 7)
    y2 = (x_value ** 3 + 7) % p
    # find the square root of y
    y = pow(y2, (p+1)//4, p)
    assert (y ** 2 % p) == y2
    # if y is odd, find even y
    if y % 2 != 0:
        y = p - y
    return coincurve.PublicKey.from_point(x_value, y)

In [12]:
import coincurve

point = lift_x(x_val)
print(point.format().hex())
print(point.point())

023905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953
(25792158117016797743484614046835601668965634494428223733945610235101691103571, 4191453718700008539754125939901750860298491226938149986069530358203844164198)


## Let's Get Familiar With The Tools!

In [13]:
from codes import parse_tx_bytes

tx = "020000000001013c29211e2e6b9a594f28fd9e8b650dd2f64e7024d398fbc4c98f3b8800cef1350000000000ffffffff0358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c95358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953eba60d00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c9530140bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de00000000"

In [14]:
print(parse_tx_bytes(tx))

AssertionError: 

In [ ]:
from pprint import pprint

In [ ]:
pprint(parse_tx_bytes(tx))

First things first, let's update the `parse_tx_bytes` method to be able to handle segwit (any version) transactions.

In [19]:
from codes import parse_compact_size, parse_input_bytes, parse_output_bytes

def parse_tx_bytes_mine(tx_hex):
  tx_bytes = bytes.fromhex(tx_hex)

  tx = {}
  ptr = 0
  tx['version'] = tx_bytes[0:4]
  ptr += 4

  if tx_bytes[ptr] == 0x00:
    assert tx_bytes[ptr+1] == 0x01
    tx['marker_flag'] = bytes([0x00, 0x01])
    ptr += 2

  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size 
  tx['inputs'] = []
  for _ in range(0, count):
    inputx, size = parse_input_bytes(tx_bytes[ptr:])
    ptr += size
    tx['inputs'].append(inputx)
 
  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size
  tx['outputs'] = []
  for _ in range(0, count):
    outputx, size = parse_output_bytes(tx_bytes[ptr:])
    ptr += size
    tx['outputs'].append(outputx)

  if 'marker_flag' in tx:
    tx['witnesses'] = []
    for _ in range(0, len(tx['inputs'])):
        witness, size = parse_witness_bytes(tx_bytes[ptr:])
        ptr += size
        tx['witnesses'].append(witness)

  tx['locktime'] = tx_bytes[ptr:]
  return tx

In [ ]:
tx_bytes = bytes.fromhex(tx)
ptr = 4
print(tx_bytes.hex())
print(tx_bytes[ptr:].hex())

In [ ]:
pprint(parse_tx_bytes_mine(tx))

In [20]:
def parse_witness_bytes(data):
    count, size = parse_compact_size(data)
    ptr = size
    witnesses = []
    for _ in range(0, count):
        witlen, size = parse_compact_size(data[ptr:])
        ptr += size
        witnesses.append(data[ptr:ptr+witlen])
        ptr += witlen

    return witnesses, ptr

In [ ]:
txdata = parse_tx_bytes_mine(tx)

In [ ]:
assert len(txdata['witnesses'][0]) == 1

In [ ]:
print(txdata['witnesses'][0][0].hex())

In [ ]:
sig = txdata['witnesses'][0][0]

In [ ]:
print(sig.hex())

In [ ]:
print(len(sig))

## Validating a Signature

What message did this signature sign?

Finding the "sighash" of that message. 

Take the bitcoin transaction that the signature was embedded into. From this transaction, we'll create a hash.

That hash of the bitcoin transaction "message digest" which is the hash that the signature "signed".

In [ ]:
print(tx)

In [ ]:
pprint(txdata)

## Find the SigMsg for a transaction

Here we use the BIP341 Signature Validation Rules (https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki#user-content-Common_signature_message) Common signature message definition to calculate the SIGHASH_DEFAULT for a given transaction.

We'll use this to compute the sighash for this input, which we need to validate the signature.

In [18]:
from hashlib import sha256
from codes import size_compact_size

def sigmsg_default(tx_hex, input_index, annex_bytes, amounts_bytes, scriptpubkeys_bytes, ext_flag):
    txdata = parse_tx_bytes_mine(tx_hex)
    
    result = b''
    result += bytes([0x00]) # sighash_flag
    result += txdata['version']
    result += txdata['locktime']

    all_input_outpoints = b''
    for inp in txdata['inputs']:
        # txid || vout
        all_input_outpoints += inp['txid'] + inp['vout']    
    sha_prevouts = sha256(all_input_outpoints).digest()
    result += sha_prevouts
    
    sha_amounts = sha256(b''.join(amounts_bytes)).digest()        
    result += sha_amounts
    
    spks = b''
    for spk in scriptpubkeys_bytes:
        spks += size_compact_size(len(spk)) + spk
    sha_scriptpubkeys = sha256(spks).digest()
    result += sha_scriptpubkeys
    
    all_sequences = [ i['sequence'] for i in txdata['inputs'] ]
    sha_sequences = sha256(b''.join(all_sequences)).digest()
    result += sha_sequences
    
    all_outputs = b''
    for o in txdata['outputs']:
        # amount || compact_size(scriptpubkey) || scriptpubkey
        all_outputs += o['amount']
        all_outputs += size_compact_size(len(o['scriptPubKey']))
        all_outputs += o['scriptPubKey']
    sha_outputs = sha256(all_outputs).digest()
    result += sha_outputs
    
    # data about this input
    annex_present = 1 if annex_bytes else 0
    spend_type = 2 * ext_flag + annex_present
    result += (spend_type).to_bytes(1, 'little')
    result += (input_index).to_bytes(4, 'little')
    
    if (annex_bytes):
        size_bytes = size_compact_size(len(annex_bytes))
        sha_annex = sha256(size_bytes + annex_bytes).digest()
        result += sha_annex
    
    # data about this output
    # noop for SIGHASH_DEFAULT
    
    assert len(result) <= 206
    return result

In [21]:
spent_from_tx = parse_tx_bytes_mine("02000000000101d79ca7455775b3fc7bf65eae536faa22e6248b863247cee30765fabb4fafadb80100000000ffffffff02dcab0e00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953944be70400000000225120dc2e293b7717ece7898fefc8fd7f6da13e8fb56efac0bc3bd75183304da6ad1701409dab8ddd9c703d1acb6736bc5260b23e64999cb130b9b3ca6c2e69aa084073302b66f0ecb5b4a191618951c022da2c429ac3638613c43bbb88a761634c2cb85300000000")

print(spent_from_tx['outputs'][0])

{'amount': b'\xdc\xab\x0e\x00\x00\x00\x00\x00', 'scriptPubKey': b'Q 9\x05\xd8\x1bE{B\xe8\xd5\xeb\x1a\xe5(\x0c\x83\xc7O\xbd\xd2\n.@\x1c\x00w\xbc\xe5Ay\t\xc9S'}


In [22]:
amounts_bytes = [spent_from_tx['outputs'][0]['amount']]
print("amounts:", [x.hex() for x in amounts_bytes])

scriptpubkeys_bytes = [spent_from_tx['outputs'][0]['scriptPubKey']]
print("scriptpubkeys:", [x.hex() for x in scriptpubkeys_bytes], "\n")

ext_flag = 0
annex_bytes = None
input_index = 0

sigmsg = sigmsg_default(tx, input_index, annex_bytes, amounts_bytes, scriptpubkeys_bytes, ext_flag)

print("sigmsg:", sigmsg.hex())

amounts: ['dcab0e0000000000']
scriptpubkeys: ['51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953'] 

sigmsg: 000200000000000000002b9b64a25d11192d251c2e088e7cbc65889963ff67244102b5fc7cb1b4feb51c4eb25d1adc8a44ed6dd54bca7d3d790a7b9e66ef9a8416836882124ad71bb507d41f17121f975dfae4cb8a052b3e34576e1c272bf7e17dfe484be8d1f96317ad95131bc0b799c0b1af477fb14fcf26a6a9f76079e48bf090acb7e8367bfd0e5cdaded8d0fdd6bcb42703ac71156c5d658a47acaa0b90924087e48059d5f1c30000000000


## Computing the Sighash

Now that we have the sighash message, we can calculate the actual "sighash" for the signature.

The `sighash` for Taproot key path spending signature valdiation is as follows:

```
sighash = tag_hash(b'TapSighash', bytes([0x00]) + sigmsg)
```

We'll need to implement the `tag_hash` function, then pass our `sigmsg` into this to find the `sighash`.

In [23]:
from hashlib import sha256

def tag_hash(tag, data_bytes):
    taghash = sha256(tag).digest()
    return sha256(taghash + taghash + data_bytes).digest()

In [24]:
sighash = tag_hash(b'TapSighash', bytes([0x00]) + sigmsg)
print(sighash.hex())

c09a12ac690db9c044c0df972aec9f574cf12677b80b1b82e910bf9e16f3b115


## Verifying the Sighash

Now that we've got the `sighash` we're finally ready to verify the signature.

For this, we'll need the `verify` function, which is defined in BIP340.

In [ ]:
# there are three inputs to the verify function: pubkey, message, signature
pk = x_val
m = sighash
sig = sig

print(x_val)

In [25]:
from codes import n

def verify(x_val, m, sig):
    P = lift_x(x_val)
    assert len(sig) == 64
    r = int.from_bytes(sig[:32], 'big')
    if r >= p:
        return False
    s = int.from_bytes(sig[32:], 'big')
    if s >= n:
        return False
    
    P_bytes = (x_val).to_bytes(32, 'big')
    challenge_hash = tag_hash(b'BIP0340/challenge', sig[:32] + P_bytes + m)
    e = int.from_bytes(challenge_hash, 'big') % n
    
    # e * P
    eP = P.multiply(e.to_bytes(32, 'big'))
    # s * G
    sG = coincurve.PrivateKey.from_int(s).public_key
    neg_eP = invert_point(eP)
    
    # R = sG - eP = sG + -eP
    R = coincurve.PublicKey.combine_keys([sG, neg_eP])
    
    if not has_even_y(R):
        return False
    
    return R.point()[0] == r

In [26]:
def invert_point(pt):
    x, y = pt.point()
    yneg = p - y
    return coincurve.PublicKey.from_point(x, yneg)

In [27]:
def has_even_y(pt):
    return pt.point()[1] % 2 == 0

In [28]:
sig = txdata['witnesses'][0][0]
verify(x_val, m, sig)

NameError: name 'txdata' is not defined

## Signing a message

Now that we've been able to successfully verify a message, let's finish implementing the second method for Schnorr: the Sign method.

This takes a private key and a message and returns a valid signature.

```
sig = Sign(privkey, msg)
```

In [29]:
def scalar_to_pubkey(scalar_val):
    assert scalar_val != 0 and scalar_val < n
    P = coincurve.PrivateKey.from_int(scalar_val).public_key
    result = scalar_val if has_even_y(P) else n - scalar_val
    
    return result, P


def sign_msg(privkey_int, msg, nonce_int):
    d, P = scalar_to_pubkey(privkey_int)
    k, R = scalar_to_pubkey(nonce_int)
    
    bytes_R = (R.point()[0].to_bytes(32, 'big'))
    bytes_P = (P.point()[0].to_bytes(32, 'big'))
    challenge_hash = tag_hash(b'BIP0340/challenge', bytes_R + bytes_P + msg)
    e = int.from_bytes(challenge_hash, 'big') % n
    
    # ~~~BEGIN SCHNORR PART~~~
    s = (k + e * d) % n
    # ~~~ END SCHNORR PART ~~~
    
    sig = bytes_R + s.to_bytes(32, 'big')
    assert verify(P.point()[0], m, sig)
    return sig

In [30]:
privkey = 10
nonce = 5
sig = sign_msg(privkey, m, nonce)

print(sig.hex())

NameError: name 'm' is not defined

## Taproot: Exploring the Depths

We've just done a Taproot keypath spend verification (~P2PK)

Now we're going to look at how to build out scripts for taproot!

We're going to make our first taproot script. (~P2WSH)

What are the steps we need to do for this:

- figure out what scripts can spend our bitcoin
- make a tree out of them (just like christmas!)
- build a merkle root
- pick a internal pubkey
- make a tweak from the merkle root
- add the tweak to the internal pubkey to get our external pubkey
- lock money up to external pubkey

### Step 1: Figure out what scripts can spend our bitcoin

- Everyone in class should make a script!
- Each individually lock up coins to the script (single script spend)
- Send all the scripts to Lisa, and then we'll build a tree out of them

#### Writing a taproot script

nifty's script: OP_4 OP_ADD OP_8 OP_EQUAL  (requires OP_4 to unlock)

Convert to script: 
        OP_4 OP_ADD OP_8 OP_EQUAL
        54935887
        
        

In [114]:
nifty = '54935887'
casey =  'AA201ccf6c5e6212f524600fb6b20275cd6ae26dc6b812cd1f84dd3ff7d86b1937d187'
jose = '54935887'
dpp = '930200088763a820150faa5b485225f681b179f710cb169b92b401f954392eb30e677624135233f08768'
mk = '76769393010987'
damian = 'a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587'
chris = '53935987'
nate = 'a82041ef4bb0b23661e66301aac36066912dac037827b4ae63a7b1165a5aa93ed4eb87'

In [108]:
scripts = [('nifty', nifty), ('casey', casey), ('jose', jose), ('dpp', dpp), ('mk', mk), ('damian', damian), ('chris', chris), ('nate', nate)]

### Step 2: Make a tree out of your scripts

For this first round, we're just going to use a single script!

Every tree has leaves! Every script in our tree is a leaf.

Our tree is just [nifty]

In [116]:
tree = [nifty]

In [42]:
## Step 3: Build a Merkle root

In [44]:
### Step 3a: Write a function to compute a leaf (hash)

In [115]:
def make_leaf(script_bytes):
    leaf_version = 0xc0
    # leaf version + len script + script
    data = bytes([leaf_version]) + size_compact_size(len(script_bytes)) + script_bytes
    return tag_hash(b'TapLeaf', data)

In [35]:
tree_bytes = [ bytes.fromhex(x) for x in tree ]

In [117]:
leaf = make_leaf(tree_bytes[0])
print(leaf.hex())

7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6


In [109]:
for name, script in [ (a,bytes.fromhex(x)) for a,x in scripts]:
    int_leaf = make_leaf(script)
    print(name, int_leaf.hex())

nifty 7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6
casey bcf0ffea8d704f70610ddffb14a2cd74cbde7b4bf116605143f43fcdac862858
jose 7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6
dpp 7199c3d01803160ae132c34a5aaa9e5ba44725b0e6a483caa4de6804e77e2b01
mk fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93a
damian 5f305f38bd7306856e35ac66b1f0c528a76af3ecfb777c4afef45bb80b2a0a6a
chris 8ddbf7add8c35b385346ea0f1c3fa93bbfd9583c75ee097fe0c03c64cceccd0a
nate 002a01ee216a40f3d68710b87d7fb0654a7d540e9baa3f5ba5a06170b82ed08d


In this first example, where we only have one script, the "merkle root" is the leaf of the single script.

Our root = leaf.

In [118]:
root = leaf
print(leaf.hex())

7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6


### Step 4: Pick an internal pubkey

Considerations to make when picking an *internal* pubkey:

- Does it need to be spendable?
- If yes, who has the keys to it?
- If no, how can I prove it's unspendable?


In our case, our key needs to be unspendable. We need to pick a point that we don't know the private key to.

The spec (BIP341) recommends using the following algorithm (or something like it) to pick a point where the private key is unknowns.

> One example of such a point is H = lift_x(0x50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0) which is constructed by taking the hash of the standard uncompressed encoding of the secp256k1 base point G as X coordinate. In order to avoid leaking the information that key path spending is not possible it is recommended to pick a fresh integer r in the range 0...n-1 uniformly at random and use H + rG as internal key. It is possible to prove that this internal key does not have a known discrete logarithm with respect to G by revealing r to a verifier who can then reconstruct how the internal key was created.

In [112]:
# We define the nums_point as
# H(G) => sha256(uncompressed)
def nums_point(r=0):
    Gx, Gy = coincurve.PrivateKey.from_int(1).public_key.point()
    data = bytes([0x04]) + Gx.to_bytes(32, 'big') + Gy.to_bytes(32, 'big')
    h = sha256(data).digest()
    h_int = int.from_bytes(h, 'big')
    H = lift_x(h_int)
    if r > 0:
        assert r < n
        rG = coincurve.PrivateKey.from_int(r).public_key
        return coincurve.PublicKey.combine_keys([rG, H])
    return H

In [113]:
internal_pubkey = nums_point()
print(internal_pubkey.format().hex())

pubkey_bytes = internal_pubkey.point()[0].to_bytes(32, 'big')

0250929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0


### Step 5: Make a tweak from the Merkle Root

What is a tweak? Ideally, a tweak is really just a number (scalar).

We're gonna take the root and a pubkey, make a hash out of both of them and then convert this hash to a number.

We'll then use that number (scalar) as if it were a private key and find the curve point at that value.

In [62]:
def make_tweak_pubkey(pubkey, merkle_root_bytes):
    # Find the bytes for the x-value of the internal pubkey
    pubkey_bytes = pubkey.point()[0].to_bytes(32, 'big')
    
    # Compute the tweak hash (TapTweak tag + pubkey + merkle root)
    tweak_bytes = tag_hash(b'TapTweak', pubkey_bytes + merkle_root_bytes)
    # Convert that hash into a scalar (so we can make it a point)
    tweak_int = int.from_bytes(tweak_bytes, 'big')
    # Make sure our tweak is inside the "point range (n)"
    assert tweak_int < n
    
    # Multiply the tweak by the Generator Point (G) which gives us T
    T = coincurve.PrivateKey.from_int(tweak_int).public_key
    
    return T

In [76]:
T = make_tweak_pubkey(internal_pubkey, root)
print(T.format().hex())

03276d6adba4bf408222b0923821f9d6286d7d2e0750f99fab541e199c224e2b74


In [77]:
for name, script in [ (a,bytes.fromhex(x)) for a,x in scripts]:
    root = make_leaf(script)
    T = make_tweak_pubkey(internal_pubkey, root)
    print(name, T.format().hex())

nifty 03276d6adba4bf408222b0923821f9d6286d7d2e0750f99fab541e199c224e2b74
casey 02d50363f23b5712ddaf1e2f1ddbb754f95607e99e26a8983da8c44c7525c43434
jose 0341eaf5244e37c5bdf0b779a5d641cbbe0c0c5b681601e6da490e9213b940c274
dpp 02e134c26d8e2e52f1339507d00c698a840bf8dd8e6480b9b6fdc2d803805e4b50
mk 03154f265ff85914f6d7e8506663d89cf12fa2ac167946cfe97f5eec3f4a934420
damian 031dd4af1b86eae0f26b5235f33e121b82af19d25827a4b79432a3c9d52424129d
chris 0214decaa772191bdd99d46bf9ba0578f9dd97a900baf49aa55300c87cbfdd9223
nate 021410525faa15e71bb2290bb425043f220639ae89815e768ff9fcef55f017386b


Now we have our Tweak as a point. All we need to do is combine it with the internal pubkey to get an external pubkey.

### Step 6: Add Internal Pubkey to Tweak Pubkey

For some reason the spec makes everything more fancy that it needs to be.

The biggest difference is that we "promote" the internal pubkey to a x-only pubkey (even) before we add it to the tweak.

In [82]:
def make_external_pubkey(pubkey, T):    
    P = lift_x(pubkey.point()[0])
    Q = coincurve.PublicKey.combine_keys([P, T])
    return Q

In [104]:
nifty

nifty_leaf = make_leaf(bytes.fromhex(nifty))
print(internal_pubkey.format().hex())
print('nifty root:', nifty_leaf.hex())
nifty_T = make_tweak_pubkey(internal_pubkey, nifty_leaf)
print('nifty tweak', nifty_T.format().hex())
nifty_Q = make_external_pubkey(internal_pubkey, nifty_T)
print('nifty!', nifty_Q.format().hex())

0250929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0
nifty root: 7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6
nifty tweak 03276d6adba4bf408222b0923821f9d6286d7d2e0750f99fab541e199c224e2b74
nifty! 029970f40840ae66e7f420877f2b48d331eda1da215d05ec1de6bd6cd4ea45ab9f


In [162]:
for name, script in [ (a, bytes.fromhex(x)) for a,x in scripts]:
    root = make_leaf(script)
    T = make_tweak_pubkey(internal_pubkey, root)
    Q = make_external_pubkey(internal_pubkey, T)
    print(name, Q.format().hex())

nifty 029970f40840ae66e7f420877f2b48d331eda1da215d05ec1de6bd6cd4ea45ab9f
c0
casey 025a62e8ead0b9f60f5a91fd69d4044b531e086823cbab7b67ccbb312509eb9fb5
c0
jose 029970f40840ae66e7f420877f2b48d331eda1da215d05ec1de6bd6cd4ea45ab9f
c0
dpp 028df34c51844750dc133f9521285ec78c92a8ba336d435951e8f1dc68611aca58
c0
mk 03311eb2f7a8ca651980285fb6bbf2c3d548191cdb9e6ba902562fffe35fa28aab
c1
damian 03f933d817c628bb0e249a44138753de965d271fb2b347daf54b24225cac8bb1a3
c1
chris 030c2d7cf15ed86872eb9957326fe706ec54e2d100e03505538c54cae5919453ee
c1
nate 02806750c9417bbb6b017bd96a47bcf1372120310d0cee67e5eeff5cd5bf28bd0e
c0


### Step 7: Lock bitcoin up to your external pubkey! (but first make a P2TR script)

We'll take Q, drop the parity byte at the front and make this into a P2TR script.

In [125]:
def make_p2tr(Q):
    Qx_bytes = Q.point()[0].to_bytes(32, 'big')
    witness_version = 0x51
    return bytes([witness_version]) + size_compact_size(len(Qx_bytes)) + Qx_bytes


print(make_p2tr(nifty_Q).hex())
nifty_script = make_p2tr(nifty_Q).hex()

51209970f40840ae66e7f420877f2b48d331eda1da215d05ec1de6bd6cd4ea45ab9f


In [140]:
# FIXME: add `jq` to nix-shell
address_arr = !bitcoin-cli -regtest decodescript "$nifty_script" | jq -r .address
print(address_arr[0])
address = address_arr[0]

bcrt1pn9c0gzzq4enw0apqsaljkjxnx8k6rk3pt5z7c80xh4kdf6j94w0s4fwuky


In [128]:
!bitcoin-cli -regtest createwallet base58

{
  "name": "base58",
  "warning": ""
}


In [143]:
!bitcoin-cli -regtest -generate 101

{
  "address": "bcrt1q96dx4r0kp2edmy68wh36m2wgr5k9atp9t5ur0p",
  "blocks": [
    "1823d76e00da3dd3c6ace4093cdde2e5645d90a05b62b917a8ffbb37a62cc487",
    "115a3572f4c2c759badd0c34b193d4cd876e6a548da50a57ff491736be57b3f0",
    "34b1cbbd9018febe10e281282843a2cc97f3a2ead4983225997a15b024a436d0",
    "6ad02c4ab7a73645dd200cb7e196bc10d1c612f15cde5d2b63775adb33773e90",
    "5c61b32fb5c8c693a9272f5975235e38da2597778605d35326d780f3f840530f",
    "4203beed047c64453e2d89baab6bff045e14a241ce748fda1016b07721a376d9",
    "14c92870fba44c5ab33f61d237d7a6db0546397239df721f317f381134da589b",
    "76f4a9b2d8576d40e22825843789f6480d6c7e362994f2b67d11f86b5c527c74",
    "3237d66b384737982a0a68d7c894f7c1ec8416fdf13f908a43dc55ccb2e37287",
    "4c3760cd9213bbc7b662340d4e27292c619e7de56f6eb7921896eda9011754e4",
    "026a861028c20b309338008be87dfab00ba6ff699c368cea82def202bf73c7e5",
    "660220648bcb964359c93950c4e79ba8d4cafd8691b667f13afd8166bfd3be1e",
    "3015351960bb4fe7074146e69f070d37647d128073caa53566f19b

In [145]:
!bitcoin-cli -regtest sendtoaddress "$address" 1.0

d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4


In [147]:
!bitcoin-cli -regtest getrawtransaction d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4 true

{
  "txid": "d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4",
  "hash": "28b19d397de496236909ea2d2a3eb70802b31c309c4297d0ebc21d02b9273e09",
  "version": 2,
  "size": 246,
  "vsize": 165,
  "weight": 657,
  "locktime": 204,
  "vin": [
    {
      "txid": "2036ba3318a97d1afc4c5928ed207bd0338c0e0fe9f1e5f9d3a449c4999da442",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "304402207209c8815fd3682acc1e56c2b31ce81424465fea29eb5c05db11bd47260470fd02204173b179b4895ac7f4a7a5ab5811520e19e9dde41e4023e2e16a5ae211c4bc3301",
        "031cd06e632be8477b8d48ec12fa2a3658026c494976591fe013f27852a9b55cea"
      ],
      "sequence": 4294967294
    }
  ],
  "vout": [
    {
      "value": 48.99999835,
      "n": 0,
      "scriptPubKey": {
        "asm": "1 9486f5da82a06bbdea6d1534f4f700bba1265440d95d2b9f5967fadef5324a59",
        "desc": "addr(bcrt1pjjr0tk5z5p4mm6ndz560facqhwsjv4zqm9wjh86evladaafjffvsdf36rd)#3fvjrr30",
     

## Spending a Tapscript!

We should have a transaction output with bitcoin in it that's locked to your custom written script.

Now we want to create a transaction that spends that output.

My output is in transaction d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4 and the index is 1

In other words the outpoint for my bitcoins is `d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4:1`

Let's a build a transaction that spends this!

In [149]:
txid = 'd57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4'
reverse_txid = bytes.fromhex(txid)[::-1].hex()
print(reverse_txid)

amount = (1 * 10 ** 8 - 400).to_bytes(8, 'little')
print(amount.hex())

a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d5
70dff50500000000


In [150]:
!bitcoin-cli -regtest getnewaddress "" bech32m

bcrt1pywat5mnqc4pyvajq3hdmujza9dwzl3lefq26w6sdhtxfgk4wyyqqsvx823


In [152]:
!bitcoin-cli -regtest getaddressinfo bcrt1pywat5mnqc4pyvajq3hdmujza9dwzl3lefq26w6sdhtxfgk4wyyqqsvx823 | jq -r .scriptPubKey

512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100


version: 02000000
marker+flag: 0001
inputs: 01
    txid: a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d5
    vout: 01000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: 70dff50500000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
    ........ unsolved problem ........
locktime: 00000000

In [153]:
tx = "0200000001a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae210000000000"
!bitcoin-cli -regtest decoderawtransaction "$tx"

{
  "txid": "65c5ee8463feeb72112a458216e8600e5f386a685cba5b63dcaa4a18d6c51327",
  "hash": "65c5ee8463feeb72112a458216e8600e5f386a685cba5b63dcaa4a18d6c51327",
  "version": 2,
  "size": 94,
  "vsize": 94,
  "weight": 376,
  "locktime": 0,
  "vin": [
    {
      "txid": "d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 0.99999600,
      "n": 0,
      "scriptPubKey": {
        "asm": "1 23baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100",
        "desc": "addr(bcrt1pywat5mnqc4pyvajq3hdmujza9dwzl3lefq26w6sdhtxfgk4wyyqqsvx823)#rlge0tkx",
        "hex": "512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100",
        "address": "bcrt1pywat5mnqc4pyvajq3hdmujza9dwzl3lefq26w6sdhtxfgk4wyyqqsvx823",
        "type": "witness_v1_taproot"
      }
    }
  ]
}


### What's in the witness stack for a leaf spend?

We need to put into the witness stack:
 - the unlocking script
 - original leaf script
 - "control block": this has all the data you need to verify the external public key
 
 #### Nifty's Unlock Script is...
 
 Well my original leaf script

In [156]:
!bitcoin-cli -regtest decodescript "$nifty" | jq .asm

"4 OP_ADD 8 OP_EQUAL"


How do I unlock this? Great question.

asm: OP_4
unlock script: 54

In [157]:
!bitcoin-cli -regtest decodescript 54

{
  "asm": "4",
  "desc": "raw(54)#w6hewaxw",
  "type": "nonstandard",
  "p2sh": "2N4Jgqd2vSSERhUqVKPoSSK7rutzZboETGf",
  "segwit": {
    "asm": "0 e632b7095b0bf32c260fa4c539e9fd7b852d0de454e9be26f24d0d6f91d069d3",
    "desc": "addr(bcrt1qucetwz2mp0ejcfs05nznn60a0wzj6r0y2n5mufhjf5xklywsd8fsqfghky)#ay3ukd7a",
    "hex": "0020e632b7095b0bf32c260fa4c539e9fd7b852d0de454e9be26f24d0d6f91d069d3",
    "address": "bcrt1qucetwz2mp0ejcfs05nznn60a0wzj6r0y2n5mufhjf5xklywsd8fsqfghky",
    "type": "witness_v0_scripthash",
    "p2sh-segwit": "2MszUBtiXGfCHF64e2SXBA1EDFJyauhYAzH"
  }
}


1) Unlocking script: 54
2) Original leaf data??: c0 + len(nifty) + 54935887 (variable: nifty)
3) Control block

#### How do we figure out a control block?


A control block has three parts.

1) version byte
2) the x-only pubkey of the internal key
3) proof of inclusion

##### Version Byte

We're going to need Q to figure this out.

The base value for the version byte is `c0`.
With the following modification: if Q is odd, add 1 to the base value.

If Q is even (02): 0xc0
If Q is odd (03):  0xc1

In [161]:
def control_block_version_byte(Q):
    val = 0xc0
    if Q.point()[1] % 2 != 0:
        val += 1
    return bytes([val])

print(control_block_version_byte(nifty_Q).hex())

c0


##### The x-only pubkey of the internal key

This should just be the internal pubkey, 32 bytes.

In [164]:
print(internal_pubkey.format()[1:].hex())

0250929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0
50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0


###### Proof of Inclusioh

The proof of inclusion for a leaf-only spend is empty.

In [194]:
control_block = control_block_version_byte(nifty_Q) + internal_pubkey.format()[1:]

print('witness data!')
print('03')
print('01 04')
print('04 ' + nifty)
print('21 ' + control_block.hex())

witness data!
03
01 04
04 54935887
21 c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0


version: 02000000
marker+flag: 0001
inputs: 01
    txid: a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d5
    vout: 01000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: 70dff50500000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
03
    01 04
    04 54935887
    21 c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0
locktime: 00000000

In [195]:
spend_tx = '02000000000101a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030104045493588721c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac000000000'
#!bitcoin-cli -regtest decoderawtransaction "$spend_tx"

In [196]:
print(spend_tx)

02000000000101a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030104045493588721c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac000000000


In [197]:
!bitcoin-cli -regtest testmempoolaccept '["02000000000101a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030104045493588721c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac000000000"]'

[
  {
    "txid": "65c5ee8463feeb72112a458216e8600e5f386a685cba5b63dcaa4a18d6c51327",
    "wtxid": "7b50f74e105ac8a955af55172c263b0807b946a79a54120a1380b195d148b469",
    "allowed": true,
    "vsize": 105,
    "fees": {
      "base": 0.00000400
    }
  }
]
